## Discretization

In case of the column, there are several options for adapting the spatial discretization of the PDE model.
However, the two most important ones are the number of grid cells in the column (axial direction) and the particles.
Since the lumped rate model without pores does not have particles, we only need to specify axial cells `n_col`.
The default is $100$ which should work for most scenarios.


```{note}
CADET-Core by default uses a finite volume scheme for the spatial discretization.
However, we have with Version 5.0 implemented a new method, the *Discontinuous Galerkin* method which can increase speed substantially.
```

In [ ]:
from CADETProcess.processModel import ComponentSystem, Inlet, Outlet
from CADETProcess.processModel import LumpedRateModelWithoutPores, FlowSheet, Process

component_system = ComponentSystem(1)

inlet = Inlet(component_system, 'inlet')
inlet.flow_rate = 1e-6
inlet.c = 1

outlet = Outlet(component_system, 'outlet')
outlet = Outlet(component_system, "outlet")

column = LumpedRateModelWithoutPores(component_system, "column")
column.total_porosity = 0.4
column.length = 0.1  # m
column.diameter = 0.2  # m
column.axial_dispersion = 1e-9  # m^2 s^-1

flow_sheet = FlowSheet(component_system)
flow_sheet.add_unit(inlet)
flow_sheet.add_unit(outlet)
flow_sheet.add_unit(column)
flow_sheet.add_connection(inlet, column)
flow_sheet.add_connection(column, outlet)

process = Process(flow_sheet, 'process')
process.cycle_time = 2000  # s

In [ ]:
event = process.add_event('start load', 'flow_sheet.inlet.c', 1, 0)
print(event)
process.add_event('start wash', 'flow_sheet.inlet.c', 0, 60)

In [ ]:
from CADETProcess.simulator import Cadet
simulator = Cadet()

simulation_results = simulator.simulate(process)

In [ ]:
column.discretization

### High discretization

In [ ]:
column.discretization.ncol = 2000
simulation_results = simulator.simulate(process)
simulation_results.solution.outlet.outlet.plot()

### Low discretization

In [ ]:
column.discretization.ncol = 20
simulation_results = simulator.simulate(process)
simulation_results.solution.outlet.outlet.plot()

## Visualization

Additionally to the solution at the inlet and outlet of a unit operation, we can also take a look inside the column to see the peak move.

For this purpose, set the flag in the unit's `SolutionRecorder`.
Then, the `SimulationResults` will also contain an entry for the bulk.

**Note:** Since this solution is two-dimensinal (space and time), the solution can be plotted at a given position (`plot_at_location`) or a given time (`plot_at_time`).

In [ ]:
column.discretization.ncol = 100
column.solution_recorder.write_solution_bulk = True

simulation_results = simulator.simulate(process)

In [ ]:
simulation_results.solution.column.bulk.plot_at_position(0.05)
simulation_results.solution.column.bulk.plot_at_time(80)

In [ ]:
%matplotlib ipympl
from ipywidgets import interact, interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt


fig, ax = simulation_results.solution.column.bulk.plot_at_time(0)
ax.set_ylim(0, 1)
plt.tight_layout()

# Visualization
def graph_column(time=0):
    ax.clear()
    simulation_results.solution.column.bulk.plot_at_time(time, ax=ax)
    ax.set_ylim(0, 1)

style = {'description_width': 'initial'}
interact(
    graph_column,
    time=widgets.IntSlider(
        min=0, max=process.cycle_time, step=10, layout={'width': '800px'}, style=style, description='Time'
    )
)